<a href="https://colab.research.google.com/github/Tiago-Pontes/Desafio-Engenharia-de-Dados/blob/main/Hoje_gra%C3%A7as_a_Deus_deu_certo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Instalando a  biblioteca gdown*

In [1]:
!pip install gdown
!pip install --upgrade --no-cache-dir gdown

*Importando as  bibliotecas gdown e pandas*

In [2]:
import gdown
import pandas as pd

*Download com os ids direto para a VM*

In [3]:
!gdown --id 1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6
To: /content/customers_rating.csv
100% 658M/658M [00:05<00:00, 113MB/s] 


In [4]:
!gdown --id 1gLsCjaMrL91ECdThq58cZAzB9tPxG18g

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1gLsCjaMrL91ECdThq58cZAzB9tPxG18g
To: /content/movies.csv
100% 160k/160k [00:00<00:00, 73.6MB/s]


*Criando os DataFrames*

In [5]:
# Inserindo nome nas colunas do arquivo 'movies.csv'
column_names = ['Movie_Id', 'Movie_Year'] 

# Váriavel base_1 lendo em csv apartir do drive, com separação em ";", codificado em "UTF-8", sem cabeçalho, acrescentando nome as colunas.
base_1 = pd.read_csv("movies.csv", sep = ';',encoding = "UTF-8", header = None, names = column_names)

In [6]:
# Váriavel base_2 lendo o csv apartir do drive, com separação em ";", codificado em "UTF-8".
base_2 = pd.read_csv("customers_rating.csv", sep = ';',encoding = "UTF-8")

*1.1. Quantos filmes estão disponíveis no dataset?*

In [7]:
# Variável 'contag' a partir de base_1 contando as linhas da coluna 'Movie_Id'.
filmes_disp = base_1['Movie_Id'].count() 
print ('No dataset estão disponiveis {}' .format(filmes_disp),'filmes.')

No dataset estão disponiveis 4499 filmes.


---

*1.2. Qual é o nome dos 5 filmes com melhor média de avaliação?*

In [8]:
# Na variável media partindo da base_2, com agrupamento das colunas escolhidas, executando a média e contagem crescente de 5 linhas em 'Rating'.
media = base_2[['Movie_Id','Rating']].groupby('Movie_Id').mean().sort_values(by='Rating',ascending=False).head(5)

# Na variável top_five está a junção das tabelas (base_1 e media) para extrair os valores procurados de 'Movie_Year'.
top_five = pd.merge(base_1, media, how = 'inner', on = 'Movie_Id')['Movie_Year']

In [9]:
print ("Os 5 filmes com melhores médias de avaliação são:","\n","\n", top_five)

Os 5 filmes com melhores médias de avaliação são: 
 
 0    (Lord of the Rings: The Return of the King: Ex...
1                       (The Simpsons: Season 6, 1994)
2    (Ghost in the Shell: Stand Alone Complex: 2nd ...
3                               (Lost: Season 1, 2004)
4                                    (Inu-Yasha, 2000)
Name: Movie_Year, dtype: object


---

*1.3. Quais os 5 anos com menos lançamentos de filmes?*

In [10]:
# Criando uma nova variavel, separando o nome dos filme e o ano de  lançamento
new = base_1["Movie_Year"].str.split(", ",n=1, expand=True)
base_1['Movie'] = new[0]
base_1['Movie_Year'] = new[1]

# Executando um apply para retirar os parentes.
base_1['Movie_Year'] = base_1['Movie_Year'].apply(lambda x: str(x).replace(")"," "))
base_1['Movie'] = base_1['Movie'].apply(lambda x: str(x).replace("("," "))

# Convertendo 'Movie_Year'(str) para inteiro (int).
base_1['Movie_Year'] = pd.to_numeric(base_1['Movie_Year'])

In [11]:
freq = base_1.groupby(['Movie_Year']).count().head(5)
print('Os 5 anos com menos lançamentos foram: ','\n', freq['Movie_Id']) 

Os 5 anos com menos lançamentos foram:  
 Movie_Year
1915    1
1916    2
1917    1
1918    2
1920    3
Name: Movie_Id, dtype: int64


---

*1.4. Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os
filmes avaliados na última data de avaliação do dataset?*

In [12]:
# Variável 'ult_data' encontrando a última data de avaliação do dataset.
ult_data = base_2.sort_values(by='Date', ascending = False).head(1)
ult_data = ult_data['Date']

In [13]:
# Fazendo um merge da base_2 com a variável encontrada na célula anterior.
join_data = pd.merge(base_2, ult_data, left_on='Date', right_on='Date')

In [14]:
# Variável 'data_rat' criada para encontrar a média das notas na última data.
data_rat = join_data.groupby(['Movie_Id'], as_index=False)['Rating'].mean()
data_rat = data_rat.sort_values(by='Rating',ascending=False)

In [15]:
# Filtrando apenas os filmes que possuem media maior ou igual a 4.7.
dez_primeiros = data_rat.loc[data_rat['Rating'] >= 4.7]['Movie_Id'].count()

In [16]:
print("São",dez_primeiros, "filmes com notas maiores ou iguais a 4.7 na última data do dataset.")

São 195 filmes com notas maiores ou iguais a 4.7 na última data do dataset.


---

*1.5. Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e
quais as notas?*

In [17]:
# Váriavel criada com base no resiltado da pergunta anterior, apenas invertendo a sequência.

dez_ultimos = data_rat.loc[data_rat['Rating'] >= 4.7].sort_values(by='Rating', ascending = True)

In [18]:
# Junção da variável anterior com base_1 para retirar as notas e o nome dos filmes.
dez_ultimos = pd.merge(dez_ultimos, base_1, left_on='Movie_Id', right_on='Movie_Id').head(10)

In [19]:
print("Dos filmes encontrados na questão anterior, os 10 filmes com as piores notas são: ","\n",dez_ultimos)

Dos filmes encontrados na questão anterior, os 10 filmes com as piores notas são:  
    Movie_Id    Rating  Movie_Year                             Movie
0      3446  4.714286        2002                     Spirited Away
1      3551  4.750000        1977                     Pete's Dragon
2      4409  4.750000        2002   SpongeBob SquarePants: Season 3
3      1947  4.750000        2002           Gilmore Girls: Season 3
4      2585  4.750000        1994     Absolutely Fabulous: Series 2
5      1208  4.750000        1963        The Twilight Zone: Vol. 15
6      1800  4.750000        2002       An Evening With Kevin Smith
7      2129  4.750000        1960        The Twilight Zone: Vol. 41
8      4353  4.800000        2002    Curb Your Enthusiasm: Season 3
9       677  4.800000        2001              In the Mood for Love


---

*1.6. Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um
fez?*

In [20]:
 # Na variável 'customers' faço um agrupamento de  'Cust_Id' contando os votos em ordem crescente das 5 primeiras linhas.
customers = base_2.groupby(['Cust_Id']).size().sort_values(ascending=False).head()

print("Os 5 customer com mais mais avaliações de filmes são: ","\n", customers)


Os 5 customer com mais mais avaliações de filmes são:  
 Cust_Id
305344     4467
387418     4422
2439493    4195
1664010    4019
2118461    3769
dtype: int64


---